# Interact with NVIDIA FLARE System

There are several ways of interacting with NVIDIA FLARE System

* Interact via FLARE admin console
* Interact via FLARE python API
* Interact via FLARE Job CLI 


Before we dive into each interacted mode, lets prepare a job to submit. We can use the previous cifar 10 training job. 

* Install requirements
* download data
* generate job config

In [ ]:
! echo "install requirements"
! pip install -r code/requirements.txt

In [ ]:
! echo "download data"
! python  code/data/download.py

download data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:

! echo "generate job config"

%cd code

! python fl_job.py

%cd ../.

! echo "show job directory"

! tree /tmp/nvflare/jobs/workdir/fedavg

Now start a FLARE system in POC mode

And use a terminal to start the POC without admin console.

```nvflare poc start -ex admin@nvidia.com```

Then use a separate terminal 

```nvflare poc start -p admin@nvidia.com```


At this point, we assume the system is already started, we can go through the different interaction methods


## FLARE Admin Console

Now we need to use videos to show the interactive commands from Admin console. we can take a look what it looks like here 

![FLARE Admin Console](admin_console.png)



You can submit jobs, list jobs and check results, check status of sites, and abort jobs  

## Interact via FLARE python API

Another way to interact with FLARE system is using FLARE python APIs. These APIs have the equivalent functions of the Admin Commands. And they can be issued directly from notebooks.  


Let's take a look how this can be done. 


### create session

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_secure_session
 
username = "admin@nvidia.com"
workspace= "/tmp/nvflare/poc/example_project/prod_00"
admin_user_dir = os.path.join(workspace, username)

sess = new_secure_session(username=username, startup_kit_location=admin_user_dir)
print(sess.get_system_info())

#### Submit a job

In [ ]:
job_dir = "/tmp/nvflare/jobs/workdir/fedavg"
job_id = sess.submit_job(job_dir)
print(job_id + " was submitted")

In the terminal, you should see the training output, but here we would like to use API to monitor the job  

#### Monitor job
The command ```monitor_job()``` allows you to follow a job until the job is done.

By default, monitor_job() only has one required arguement, the job_id of the job you are waiting for, and the default behavior is to wait until the job is complete before returning a Return Code of JOB_FINISHED.

In order to follow along and see a more meaningful result and demonstrate some of the possibilities of using this function, the following cell contains a sample_cb() callback that keeps track of the number of times the callback is run and prints the job_meta the first three times and the final time before monitor_job() completes with every other call just printing a dot to save output space. This callback is just an example of what can be done with additional arguments and the job_meta information of the job that is being monitored. You can use logic to return a value other than True to conditionally stop monitor_job() and return MonitorReturnCode.ENDED_BY_CB.



In [ ]:
from nvflare.fuel.flare_api.flare_api import Session

def sample_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3:
            print(job_meta)
            print(cb_kwargs["cb_run_counter"])
        else:
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True

sess.monitor_job(job_id, cb=sample_cb, cb_run_counter={"count":0})

#### Get Job Meta
To get the job meta information, you can use the get_job_meta() command. The only argument required for this command is the job id for the job you are getting the meta for. The job meta information will be returned as an object.

In [ ]:

sess.get_job_meta(job_id)


#### List Jobs
To get the information for what jobs have been submitted to the server, you can use the list_jobs() command. If no arguments are included, the defaults are False for both "detailed" and "reverse".

Setting "detailed" to True will return more detailed information about each job.

Setting "reverse" to True will return the jobs in reverse order by submission time.

The arg "limit" can be set to specify the maximum number of jobs to return, with 0 or None meaning return all jobs (the default is None to show all).

The args "id_prefix" and "name_prefix" can be used to further filter the jobs returned to have an id or name beginning with the string set for the respective argument.


In [ ]:
import json

def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))

list_jobs_output = sess.list_jobs()
print(format_json(list_jobs_output))


In [ ]:
list_jobs_output_detailed = sess.list_jobs(detailed=True)
print(format_json(list_jobs_output_detailed))

#### Download Job Result
The download_job_result() command downloads the job result to the "download_dir" (this is usually set in fed_admin.json in the startup directory of the admin_user_dir used when launching the FLARE API Session, and this value is relative to the admin_user_dir). This command only has one required arguement, the job_id of the job result to download.

In [ ]:
sess.download_job_result(job_id)

#### Abort Job
If training gets stuck or there is another reason to stop a running job, you can use the abort_job() command. The abort_job() command only has one required arguement, the job_id of the job to abort, and None is returned upon successfully issuing the command.

In [ ]:
print(job_id)
sess.abort_job(job_id)

## Interact via FLARE Job CLI


#### Show variables
We can check the configuration variables with the following command

job_dir = "/tmp/nvflare/jobs/workdir/fedavg"

In [ ]:
! nvflare job show_variables -j /tmp/nvflare/jobs/workdir/fedavg

#### Submit Job from CLI
You can use the following command to directly submit job from the command line.



In [ ]:
! nvflare job submit -j /tmp/nvflare/jobs/workdir/fedavg

You use the same FLARE python APIs to monitoring the job status or stop the job.

## Summary 


So far, we have learned three different ways to interact with FLARE system. Although we used POC mode to simulate the real deployment, in production, the same interaction commands can be used in production setup.

Next, lets see how do we [monitor FLARE system](../03.4_system_monitoring/system_monitorinig.ipynb)

